In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

llm = ChatOpenAI(model="qwen-max")

model_name = "C:\\Home\\Documents\\Projects\\models\\BAAI\\bge-large-zh-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

c:\Apps\miniconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embedding_model,
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

system_prompt = (
    "你是负责问答任务的助手。"
    "使用下面检索到的上下文来回答问题。"
    "如果你不知道答案，就说你不知道。"
    "最多使用三句话并保持答案简洁。"
    "上下文：\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [4]:
response = rag_chain.invoke({"input": "物理学对马斯克有什么影响？"})

In [5]:
import textwrap

print(textwrap.fill(response["answer"], 60))

物理学为马斯克提供了一个理解复杂和反直觉事物的研究框架，比如量子力学。它教会他使用第一性原理思考问题，这是一种将事物分解
到最基本实质再进行推理的方法，对探索未知领域非常重要。这种方法帮助他在创新时能追溯问题的本质，从而产生独创性的解决方案。


# Adding chat history

In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "给定聊天记录和可能引用聊天记录中上下文的最新用户问题，"
    "制定一个无需聊天记录即可理解的独立问题。"
    "不要回答问题，只需根据需要重新表述，否则按原样返回。"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [8]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "物理学对马斯克有什么影响？"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "马斯克对这个学科有什么看法？"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(textwrap.fill(ai_msg_2["answer"], 60))

马斯克认为物理学提供了一个理想的研究框架，尤其是对于理解反直觉的新事物。他对物理学充满兴趣，并且觉得学习物理有助于探索现
实世界的本质。


In [9]:
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages


# We define a dict representing the state of the application.
# This state has the same input and output keys as `rag_chain`.
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


# We then define a simple node that runs the `rag_chain`.
# The `return` values of the node update the graph state, so here we just
# update the chat history with the input message and response.
def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


# Our graph consists only of one node:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Finally, we compile the graph with a checkpointer object.
# This persists the state, in this case in memory.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [11]:
config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": "物理学对马斯克有什么影响？"},
    config=config,
)
print(textwrap.fill(result["answer"], 60))

物理学对马斯克的影响主要体现在提供了一个研究框架，尤其是第一性原理的思维方式，帮助他理解复杂和反直觉的事物，并在创新中追
求本质，而不是仅仅依赖类推。这种思考方式对他探索新领域和解决问题至关重要。


In [12]:
result = app.invoke(
    {"input": "马斯克对这个学科有什么看法？"},
    config=config,
)
print(textwrap.fill(result["answer"], 60))

马斯克认为物理学提供了一个理想的研究框架，尤其是对于理解那些反直觉的新事物。他认为物理学的第一性原理对他来说非常重要，能
够帮助他思考复杂的问题。


In [13]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

物理学对马斯克有什么影响？
================================== Ai Message ==================================

物理学对马斯克的影响主要体现在提供了一个研究框架，尤其是第一性原理的思维方式，帮助他理解复杂和反直觉的事物，并在创新中追求本质，而不是仅仅依赖类推。这种思考方式对他探索新领域和解决问题至关重要。
================================ Human Message =================================

马斯克对这个学科有什么看法？
================================== Ai Message ==================================

马斯克认为物理学提供了一个理想的研究框架，尤其是对于理解那些反直觉的新事物。他认为物理学的第一性原理对他来说非常重要，能够帮助他思考复杂的问题。


# Agent

In [14]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever=retriever,
    name="demo_collection_retriever",
    description="Searches and returns excerpts from the Elon Musk talks."
)

tools = [tool]

In [16]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [18]:
config = {"configurable": {"thread_id": "abc345"}}

for event in agent_executor.stream(
    {"messages": [HumanMessage(content="你好！我是小明")]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

你好！我是小明
================================== Ai Message ==================================

你好，小明！有什么我可以帮助你的吗？


In [19]:
query = "物理学对马斯克有什么影响？"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

物理学对马斯克有什么影响？
================================== Ai Message ==================================
Tool Calls:
  demo_collection_retriever (call_d7e369db9b7b40dba6625a)
 Call ID: call_d7e369db9b7b40dba6625a
  Args:
    query: 物理学 对马斯克的影响
================================= Tool Message =================================
Name: demo_collection_retriever

他们说还可以回去再读，不收学
费了。
钱颖一  你先学了商科，然后学
了物理。相比经济学和商科，你
认为物理学更有趣还是更难？
马斯克  我之所以对物理和其他
科学感兴趣，是因为我想弄明白
现实世界的本质。生命的意义是什么？宇宙的本质是什么？这是
我读物理的主要动机。其实我学
商科的原因是我担心毕业后遇到
一个有商科学位的老板，如果老
板懂的我不懂，那我就没有优势
了。当然，其实也没什么。
钱颖一  你学商科是为了跟你未
来的老板有共同语言，这是当时
的动机吗？
马斯克  是的，我最怕遇上一个
我不喜欢的老板。
钱颖一  然后你就决定自己当老
板，避免这个问题？
马斯克  没错。
用物理学第一性原理思考
问题
钱颖一  你在很多场合都提到了
物理学 。你喜欢物理学 。你说
过物理学的第一性原理 （ F i r s t  
Principles）能帮助你思考复杂
的事情。能否告诉我们，或举一
些例子，为什么物理学第一性原
理对你来说这么重要？对你的生
活和思维方式到底有什么影响？
马斯克  要知道，想理解那些反
直觉的新事物，我觉得物理学提
供了一个最理想的研究框架。比
如说， 量子力学就是违背直觉的，
现实世界似乎并不是那样运转，
而事实就是，并可以通过实验高
度精确地验证。物理学之所以能

书？你最喜欢读什么方面的书？


In [21]:
query = "根据访谈记录，他对这个学科有什么看法？重新搜索"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

根据访谈记录，他对这个学科有什么看法？重新搜索
================================== Ai Message ==================================
Tool Calls:
  demo_collection_retriever (call_208ac120844445fcb78ee5)
 Call ID: call_208ac120844445fcb78ee5
  Args:
    query: 马斯克 对 物理学 的 看法
================================= Tool Message =================================
Name: demo_collection_retriever

他们说还可以回去再读，不收学
费了。
钱颖一  你先学了商科，然后学
了物理。相比经济学和商科，你
认为物理学更有趣还是更难？
马斯克  我之所以对物理和其他
科学感兴趣，是因为我想弄明白
现实世界的本质。生命的意义是什么？宇宙的本质是什么？这是
我读物理的主要动机。其实我学
商科的原因是我担心毕业后遇到
一个有商科学位的老板，如果老
板懂的我不懂，那我就没有优势
了。当然，其实也没什么。
钱颖一  你学商科是为了跟你未
来的老板有共同语言，这是当时
的动机吗？
马斯克  是的，我最怕遇上一个
我不喜欢的老板。
钱颖一  然后你就决定自己当老
板，避免这个问题？
马斯克  没错。
用物理学第一性原理思考
问题
钱颖一  你在很多场合都提到了
物理学 。你喜欢物理学 。你说
过物理学的第一性原理 （ F i r s t  
Principles）能帮助你思考复杂
的事情。能否告诉我们，或举一
些例子，为什么物理学第一性原
理对你来说这么重要？对你的生
活和思维方式到底有什么影响？
马斯克  要知道，想理解那些反
直觉的新事物，我觉得物理学提
供了一个最理想的研究框架。比
如说， 量子力学就是违背直觉的，
现实世界似乎并不是那样运转，
而事实就是，并可以通过实验高
度精确地验证。物理学之所以能

49

# Put it together

### RAG with chat history

In [ ]:
from uuid import uuid4
from langchain.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


# Load documents
loader = PyPDFLoader(file_path="../data/对话马斯克.pdf")
docs = loader.load()

# Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

# Generate embeddings and store
client = QdrantClient(host="localhost", port=6333)
if not client.collection_exists("demo_collection"):
    client.create_collection(
        collection_name="demo_collection",
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
    )
vector_store = QdrantVectorStore(
    client=client, collection_name="demo_collection", embedding=embedding_model
)
ids = [str(uuid4()) for _ in range(len(all_splits))]
vector_store.add_documents(documents=all_splits, ids=ids)

# Create retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
contextualize_q_system_prompt = (
    "给定聊天记录和可能引用聊天记录中上下文的最新用户问题，"
    "制定一个无需聊天记录即可理解的独立问题。"
    "不要回答问题，只需根据需要重新表述，否则按原样返回。"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Create RAG chain
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# We define a dict representing the state of the application.
# This state has the same input and output keys as `rag_chain`.
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


# We then define a simple node that runs the `rag_chain`.
# The `return` values of the node update the graph state, so here we just
# update the chat history with the input message and response.
def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


# Our graph consists only of one node:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Finally, we compile the graph with a checkpointer object.
# This persists the state, in this case in memory.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
import textwrap

config = {"configurable": {"thread_id": "abc123"}}

result = app.invoke(
    {"input": "物理学对马斯克有什么影响？"},
    config=config,
)
print(textwrap.fill(result["answer"], 60))

result = app.invoke(
    {"input": "马斯克对这个学科有什么看法？"},
    config=config,
)
print(textwrap.fill(result["answer"], 60))

物理学为马斯克提供了一个研究框架，帮助他理解反直觉的新事物，并通过第一性原理思考问题，这对于了解新事物和探索未知领域非常
重要。此外，他认为物理课程应该更注重传递公式的实际意义及物理的美妙，而不仅仅是记忆公式。
马斯克认为物理学提供了一个理想的研究框架，尤其是对于理解那些反直觉的新事物，比如量子力学。他对物理学很感兴趣，因为它帮助
他思考复杂的事情，并通过第一性原理来解决问题。


In [25]:
chat_history = app.get_state(config).values["chat_history"]
for message in chat_history:
    message.pretty_print()

================================ Human Message =================================

物理学对马斯克有什么影响？
================================== Ai Message ==================================

物理学为马斯克提供了一个研究框架，帮助他理解反直觉的新事物，并通过第一性原理思考问题，这对于了解新事物和探索未知领域非常重要。此外，他认为物理课程应该更注重传递公式的实际意义及物理的美妙，而不仅仅是记忆公式。
================================ Human Message =================================

马斯克对这个学科有什么看法？
================================== Ai Message ==================================

马斯克认为物理学提供了一个理想的研究框架，尤其是对于理解那些反直觉的新事物，比如量子力学。他对物理学很感兴趣，因为它帮助他思考复杂的事情，并通过第一性原理来解决问题。


In [26]:
chat_history

[HumanMessage(content='物理学对马斯克有什么影响？', additional_kwargs={}, response_metadata={}, id='34bef3a5-6bb6-492f-94db-773a58e86ca7'),
 AIMessage(content='物理学为马斯克提供了一个研究框架，帮助他理解反直觉的新事物，并通过第一性原理思考问题，这对于了解新事物和探索未知领域非常重要。此外，他认为物理课程应该更注重传递公式的实际意义及物理的美妙，而不仅仅是记忆公式。', additional_kwargs={}, response_metadata={}, id='0efaa2b4-b466-42bf-a66e-6cbcf59debc0'),
 HumanMessage(content='马斯克对这个学科有什么看法？', additional_kwargs={}, response_metadata={}, id='c08713a2-d0d4-430e-b13e-c751308747b9'),
 AIMessage(content='马斯克认为物理学提供了一个理想的研究框架，尤其是对于理解那些反直觉的新事物，比如量子力学。他对物理学很感兴趣，因为它帮助他思考复杂的事情，并通过第一性原理来解决问题。', additional_kwargs={}, response_metadata={}, id='a7cc273f-f4f6-4afc-a791-558376ce9e5b')]

### Agentic RAG with chat history

In [29]:
from uuid import uuid4
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


# Load documents
loader = PyPDFLoader(file_path="../data/对话马斯克.pdf")
docs = loader.load()

# Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

# Generate embeddings and store
client = QdrantClient(host="localhost", port=6333)
if not client.collection_exists("demo_collection"):
    client.create_collection(
        collection_name="demo_collection",
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
    )
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embedding_model
)
ids = [str(uuid4()) for _ in range(len(all_splits))]
vector_store.add_documents(documents=all_splits, ids=ids)

# Create retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Create tool
tool = create_retriever_tool(
    retriever=retriever,
    name="demo_collection_retriever",
    description="Searches and returns excerpts from the Elon Musk talks."
)
tools = [tool]

# Add chat history
memory = MemorySaver()

# Create agent
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [30]:
config = {"configurable": {"thread_id": "abc456"}}

query = "物理学对马斯克有什么影响？"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

物理学对马斯克有什么影响？
================================== Ai Message ==================================
Tool Calls:
  demo_collection_retriever (call_83753c37cd8441d1a99c6a)
 Call ID: call_83753c37cd8441d1a99c6a
  Args:
    query: physics influence on Musk
================================= Tool Message =================================
Name: demo_collection_retriever

他们说还可以回去再读，不收学
费了。
钱颖一  你先学了商科，然后学
了物理。相比经济学和商科，你
认为物理学更有趣还是更难？
马斯克  我之所以对物理和其他
科学感兴趣，是因为我想弄明白
现实世界的本质。生命的意义是什么？宇宙的本质是什么？这是
我读物理的主要动机。其实我学
商科的原因是我担心毕业后遇到
一个有商科学位的老板，如果老
板懂的我不懂，那我就没有优势
了。当然，其实也没什么。
钱颖一  你学商科是为了跟你未
来的老板有共同语言，这是当时
的动机吗？
马斯克  是的，我最怕遇上一个
我不喜欢的老板。
钱颖一  然后你就决定自己当老
板，避免这个问题？
马斯克  没错。
用物理学第一性原理思考
问题
钱颖一  你在很多场合都提到了
物理学 。你喜欢物理学 。你说
过物理学的第一性原理 （ F i r s t  
Principles）能帮助你思考复杂
的事情。能否告诉我们，或举一
些例子，为什么物理学第一性原
理对你来说这么重要？对你的生
活和思维方式到底有什么影响？
马斯克  要知道，想理解那些反
直觉的新事物，我觉得物理学提
供了一个最理想的研究框架。比
如说， 量子力学就是违背直觉的，
现实世界似乎并不是那样运转，
而事实就是，并可以通过实验高
度精确地验证。物理学之所以能

4

In [31]:
query = "他对这个学科有什么看法？"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

他对这个学科有什么看法？
================================== Ai Message ==================================

马斯克对物理学持有非常积极的看法。他认为物理学提供了一个强大的框架，特别是通过运用第一性原理来思考问题，这对于理解那些违背直觉的新概念非常重要。他觉得这种方法是探索未知领域和了解新事物的关键。

同时，他也指出，在日常生活中人们倾向于使用类推的方法来做决策，这在大多数情况下是足够有效的，因为它节省了大量思考的时间。但是，如果要进行创新，就需要回到最基本的原则上，即采用第一性原理的思维方式去挑战现有的假设，这样才能产生真正的原创思想。

此外，马斯克对于物理教育也有自己的见解。他支持在学校里开设更多的物理课程，但同时也批评当前的教学方式往往侧重于让学生记忆公式，而没有充分解释这些公式的实际意义及其所代表的美妙现实世界。他建议物理教学应该更加注重实践与问题解决，这样学生才能更好地理解和欣赏物理学的奥秘。

总而言之，马斯克认为物理学不仅是一门科学学科，更是一种能够激发创造力、推动技术革新和解决复杂问题的强大工具。他对物理学的热情也体现在他所创立的公司以及他对未来科技发展的愿景中。


In [33]:
query = "根据访谈记录，他对这个学科有什么看法？重新搜索"

for event in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

根据访谈记录，他对这个学科有什么看法？重新搜索
================================== Ai Message ==================================
Tool Calls:
  demo_collection_retriever (call_6d9e617274fd43ae838d61)
 Call ID: call_6d9e617274fd43ae838d61
  Args:
    query: 他对这个学科的看法
================================= Tool Message =================================
Name: demo_collection_retriever

猎各个科目。很多创新发明都是
跨学科的成果。我们的知识储备
越来越庞大，所以必须能够融会
贯通。有人精通一个领域，而不
了解其他领域，如果你能把不同
领域的知识结合在一起，就有机
会创造出超常成果，这里有大把
的创新机会。所以我鼓励大家尽
可能广泛地学习各个科目。对于
工科学生，我建议去学一点经济
学，学点文学，或者其他领域。
我建议，在有兴趣的前提下，大
家可以学习每个领域的基础知识，
然后思考一下如何将不同领域的知识融会贯通。这样很容易产生
奇思妙想。
钱颖一  这是我们在清华经管学
院所做的事情。我们称之为 “通
识教育”。在清华大学层面，也
越来越重视通识教育，而不是过
早进行专业教育。你认为这很重
要？
马斯克 我觉得要对所有领域有一
个大致的了解是很重要的。即使
要做专才，也至少要精通两个领
域，那样的话就可以把两个领域
的知识相结合，这里面就蕴含大
量的机会。
钱颖一  中国今年兴起了创业热
潮。很多清华学生希望创立中国的
特 斯 拉、SolarCity、PayPal，
等等。你对这些学生有什么建议？
在大学期间就创业是件好事吗，还
是读研究生期间？没有创立企业
前，对于一个创业者来说，应该具
备什么品质？你能给予这些年轻一
代什么建议吗？
马斯克 对于发现社会上的需求，
如果你发觉这确实是大

In [34]:
agent_executor.checkpointer.get(config=config)

{'v': 1,
 'ts': '2024-11-12T06:46:54.341724+00:00',
 'id': '1efa0c1e-76a6-679d-800b-def86a0a7128',
 'channel_values': {'messages': [HumanMessage(content='物理学对马斯克有什么影响？', additional_kwargs={}, response_metadata={}, id='9e6bbbcd-c495-475d-a5d9-ce6ba34dc530'),
   AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_83753c37cd8441d1a99c6a', 'function': {'arguments': '{"query": "physics influence on Musk"}', 'name': 'demo_collection_retriever'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 192, 'total_tokens': 215, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ae52ec4b-037c-4f0b-aed4-95143d85306b-0', tool_calls=[{'name': 'demo_collection_retriever', 'args': {'query': 'physics influence on Musk'}, 'id': 'call_83753c37cd8441d1a99c6a', 'type': 'tool_call'}], usage_